# Clustering Overview

This example notebook will demonstrate Howso Engine’s ability to calculate pairwise distances for the training cases. This capability is based on Howso Engine’s underlying instance-based learning platform. Distances can be used for a variety of use cases, in particular, for clustering algorithms, such as HDBSCAN. Howso Engine’s feature reduction capabilities will also be used to demonstrate how to eliminate features for clustering. 

In [1]:
import os

import numpy as np
import pandas as pd
import plotly.io as pio
from plotly.subplots import make_subplots
from sklearn.cluster import HDBSCAN
from sklearn.datasets import make_blobs
from sklearn.metrics import adjusted_rand_score

from howso import engine
from howso.utilities import infer_feature_attributes

pio.renderers.default = os.getenv("HOWSO_RECIPE_RENDERER", "notebook")

## Generate a simple dataset with noise

A 2-D blob is generated along with an extra noise column. The noise column is used to demonstrate feature reduction.


In [2]:
blobs = make_blobs(n_samples=100, n_features=2, centers=[(1,0), (1,5), (10,2)], random_state=0)
df = pd.DataFrame(blobs[0], columns=['x', 'y'])
df['target'] = blobs[1]

# Noise
noise = np.random.uniform(df['y'].min(), df['y'].max(), len(df))
df['noise'] = noise

## Build Howso Engine model

In [3]:
# Infer features using dataframe format
features = infer_feature_attributes(df)

# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)

# Create the Trainee
t = engine.Trainee(
    features=features,
    overwrite_existing=True
)

# Train
t.train(df)

# Targeted Analysis
t.analyze(context_features=context_features, action_features=action_features)

The following parameters from configuration file will override the Amalgam parameters set in the code: {'trace'}


# Feature Reduction

In this example, since we have a target variable, we can inspect the feature contributions and feature mean decrease in accuracy to understand how important each feature is.

In [4]:
t.react_into_trainee(context_features=context_features, action_feature=action_features[0], contributions_robust = True, mda_robust=True)
robust_feature_contributions = t.get_prediction_stats(action_feature=action_features[0], robust=True, stats=['contribution', 'mda'])
robust_feature_contributions

,noise,x,y
mda,-0.056305,0.069816,0.133268
contribution,0.093983,0.343970,0.403151


We can see that the noise column not only contributes less to the prediction than the other variables, but it also contributes little to the accuracy. This is expected from the noise column and demonstrates how unimportant features can be highlighted with Howso Engine.

# Feature Removal

Unuseful features like the noise feature can reduce the quality of the clustering so we will remove them in this example. Howso Engine allows use to easily remove the features from the Trainee.

In [5]:
# Remove the noise feature
t.remove_feature('noise')

# update context_features
context_features.remove('noise')

# Reanalyze the remaining features
t.analyze(context_features=context_features, action_features=action_features)

## Get pairwise distances

By directly using Howso Engine's pairwise distances, we can take advantage of all the benefits of Howso Engine, including the ease of preprocessing and categorical variable handling.

In [6]:
# pairwise distances can be obtained with a single function call
x = t.get_distances()

In [7]:
x['distances']

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,8.284045,0.116400,7.982496,8.606475,1.757592,0.780267,8.583305,4.666778,2.436646,...,1.460663,4.806385,4.289278,7.702400,0.575794,5.892589,8.451130,5.122463,4.645459,4.888756
1,8.284045,0.000000,8.366504,2.321748,2.248762,7.101197,8.241792,2.899847,8.829193,10.343308,...,6.934806,10.198379,10.378473,2.794453,8.370649,11.370238,3.351249,10.018443,9.032376,9.371425
2,0.116400,8.366504,0.000000,8.084320,8.707290,1.873913,0.712933,8.688295,4.758767,2.420678,...,1.515131,4.872426,4.340745,7.807249,0.490096,5.947996,8.558777,5.196992,4.733937,4.973350
3,7.982496,2.321748,8.084320,0.000000,0.649777,6.467253,8.160412,0.780537,7.333296,9.654352,...,6.861136,8.796684,9.135760,0.547372,8.224047,9.869857,1.081944,8.519043,7.555107,7.862945
4,8.606475,2.248762,8.707290,0.649777,0.000000,7.103816,8.766995,0.683179,7.944048,10.299615,...,7.459765,9.416088,9.768564,1.118363,8.836775,10.476145,1.190838,9.127439,8.167772,8.471220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5.892589,11.370238,5.947996,9.869857,10.476145,5.366699,6.651383,10.071170,2.565946,4.271304,...,6.838207,1.254177,1.645496,9.363567,6.419082,0.000000,9.669344,1.363858,2.352688,2.023840
96,8.451130,3.351249,8.558777,1.081944,1.190838,6.831904,8.714706,0.520412,7.185150,9.913210,...,7.450221,8.677498,9.107474,0.890934,8.751123,9.669344,0.000000,8.341878,7.413800,7.693468
97,5.122463,10.018443,5.196992,8.519043,9.127439,4.294420,5.889508,8.733889,1.202102,4.033787,...,5.857410,0.558431,1.399096,8.016758,5.678507,1.363858,8.341878,0.000000,0.991567,0.661174
98,4.645459,9.032376,4.733937,7.555107,8.167772,3.568995,5.396410,7.791859,0.232864,4.051871,...,5.177163,1.276395,1.866800,7.059749,5.209171,2.352688,7.413800,0.991567,0.000000,0.358115


# Cluster

In [8]:
hdb = HDBSCAN(min_cluster_size = 15, min_samples=10, metric='precomputed').fit(x['distances'])
hdbscan_labels = hdb.labels_
df["hdbscan_labels"] = hdbscan_labels

fig = make_subplots(rows=1, cols=2, subplot_titles=["True Clusters", "HDBSCAN Clusters"])
for label, group in df.groupby("target"):
    fig.add_scatter(x=group.x, y=group.y, col=1, row=1, mode="markers", name=label)
for label, group in df.groupby("hdbscan_labels"):
    fig.add_scatter(
        x=group.x, y=group.y, col=2, row=1, mode="markers", name=label,
        marker_color=fig.layout["template"]["layout"]["colorway"][label],
        showlegend=False
    )
fig.update_layout(width=1000)
fig.show()

# Results

We calculate the performance of the clustering by measuring the similarity of the clusters compared to the original labels.

In [9]:
dist_contribution_acc = adjusted_rand_score(df['target'], hdbscan_labels)
print(f'clustering accuracy score: {dist_contribution_acc}')

clustering accuracy score: 0.9110470233524972
